このスクリプトは、"C:\Users\KeikoGolden\TokyoOffice\SkillDictionary\Python\FurtherFilter\ skill_normalization_expanded_Copy.ipynb"　から拡張したもの。

1. Noamalize using skill_normalization.py
2. Option 2 - Igonore This Section
3. 02/29/2023 - Ignore This Section
4. 02/09/2023 - No2 Spacy
5. 02/11/2023 - SkyHive vectorizer
6. 02/11/2023 - No2 Difflib
7. 02/11/2023 Transformer Bert

# Normalize using skill_normalization.py

In [1]:
import json
import requests
import pandas as pd
from typing import List

In [2]:
def normalize_v3(skill_list, k=5):
    response = requests.post(
    url = 'https://testazure-ml-api.skyhive.io/job-title-normalization/titles_v3',
    json = {"titles": skill_list, "k": k}
)
    return response.json()

In [3]:
from tqdm import tqdm
def main_v3(skills: List[str]):
    responses = []

    for skill in tqdm(skills):
        try:
            response = normalize_v3([skill])
        except:
            response = None
        responses.append(response)
    return responses

In [4]:
file_path = "C:/Users\KeikoGolden/ComputationalLiguistics/Annotaion/bn_nb_s/PythonTocheckModel/soc_cleanup.xlsx"

In [5]:
df = pd.read_excel(file_path)
df.head(1)

,major,minor,detailed,major_title,minor_title,occupation_title_working,final_title_post_processed,definition
0,11-0000,11-1000,11-1011,Management Occupations,Top Executives,Chief Executives,chief executive,NaN


In [6]:
occupation_title = df['final_title_post_processed']
occupation_title

0                                         chief executive
1                                         general manager
2                                      operations manager
3                                              legislator
4                                     advertising manager
                              ...                        
1453    lathe and turning machine tool setters, operat...
1454    metal and plastic lathe and turning machine to...
1455    metal and plastic lathe and turning machine to...
1456    metal and plastic lathe and turning machine to...
1457    lathe and turning machine tool setters, operat...
Name: final_title_post_processed, Length: 1458, dtype: object

In [ ]:
responses = main_v3(occupation_title)
responses

In [ ]:
df['normalized'] = responses

In [ ]:
df.to_excel('normalized_soc_cleanup.xlsx')

In [ ]:
df.to_parquet('normalized_soc_cleanup.parquet')

In [ ]:
results_df = pd.read_excel('normalized_soc_cleanup.xlsx')

In [ ]:
results_df.normalized.info()

In [ ]:
import regex as re
jt_list = []
for result in results_df.normalized[:5]:
    regex1 = r"""Keywords': '(.+)', 'Results'"""
    jt = re.findall(regex1, result)
    #jt_list.append(jt)
    jt_list += jt
print(jt_list)

In [ ]:
import regex as re
detail_jt_list = []
for results in results_df.normalized:
    regex2 = r"""categoryName': '([A-Za-z ]+)', 'score'"""
    jt_normalized = re.findall(regex2, str(results))
    detail_jt_list.append(jt_normalized)

In [ ]:
detail_jt_list

In [ ]:
import regex as re
score_list = []
for results in results_df.normalized:
    regex3 = r"""categoryName': '([A-Za-z ]+)', score': '([0-9].[0-9]+)','id'"""
    score = re.findall(regex3, str(results))
    score_list.append(score)

In [ ]:
df['normalized'] = detail_jt_list

In [ ]:
df.head(10)

In [ ]:
exploded_df = df.explode('normalized')
exploded_df

In [ ]:
exploded_df.to_excel('normalized_soc_cleanup_exploded.xlsx')

# Option 2

In [ ]:
responses = main_v3(occupation_title)
responses

In [ ]:
df['normalized'] = responses

In [ ]:
df = df.dropna(subset=['normalized']).copy()

In [ ]:
#option 1

for normalized in df.normalized:
    output = [result["Keywords"]] 
    for result in results:   
        print(result['categoryName'])
        print(result['score'])
    

In [ ]:
#option 2

import csv

results = df['normalized']

with open("normalized_job_title.csv", "w", encoding="utf-8") as skill_fd:    
    skill_fd.write("\ufeff")
    skill_csv = csv.writer(skill_fd)
    skill_csv.writerow(["source", "normalised_title", "score"])
    for result in results:
        output = [result["Keywords"]]  
        for skill in result["Results"]: 
            output.append(skill["categoryName"])   
            output.append(skill["score"])           
        skill_csv.writerow(output)

In [ ]:
df3 = pd.read_csv('normalized_job_title.csv')
df3

In [ ]:
normalized_list

In [ ]:
df['normalized'] = normalized_list

In [ ]:
df

In [ ]:
exploded_df = df.explode('normalized')
exploded_df

In [ ]:
exploded_df.to_excel('normalized_soc_cleanup_exploded.xlsx')

# 02/29/2023 Ignore This Section

初期に抽出できなかった（トップ５マッチがなかった）ジョブタイトルをノーマライズ。

In [ ]:
import json
import requests
import pandas as pd
from typing import List

In [ ]:
df = pd.read_excel('soc_normalized_titles_matches_02092023.xlsx')
df

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')
tokenized_sents = []
for job_title in df.Column1:
    doc = nlp(job_title)
    #tokenized_sents = []
    sents = []
    for sent in doc.sents:
        sents.append(sent.text)
        tokens = [i.text for i in sent]
        tokenized_sents.append(tokens)

In [ ]:
tokenized_sents

In [ ]:
import boto3

# to select an aws profile if necessary
profile_name = ""
boto3.setup_default_session(profile_name=profile_name)

In [ ]:
#endpoint3から拡張

import boto3
import json
from typing import *

region = "ca-central-1"
vectorizer_name = "vectorizer-cpu-2"
sagemaker_runtime = boto3.client("sagemaker-runtime", region)

def vectorize_remote(
        sents: List[List[str]],
        offsets: List[List[List[int]]]) -> List[List[List[float]]]:
    data = {"sents": sents, "offsets": offsets}
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=vectorizer_name,
        Body=bytes(json.dumps(data), encoding="utf-8"),
        ContentType='application/json'

    )
    output = response['Body'].read().decode('utf-8')
    output = json.loads(json.loads(output)[0])
    return output

In [ ]:
"""
[
    [0, 24], # entire sentence --> vector
    [15, 18], # SKILL   --> vector
    [4, 5],  # noun --> vector
    [10, 11] # verb --> vector
    [20, 26] # SKILL --> vector

]
"""

In [ ]:
offsets = [[[0, len(sent)]] for sent in tokenized_sents]
offsets

In [ ]:
vectors = vectorize_remote(
    tokenized_sents,
    offsets
)

vectors

In [ ]:
sagemaker_faiss_name = "semantic-search-cpu-2"

def get_matches(vectors: List[List[List[float]]]) \
        -> List[List[List[float]]]:
    """Get semantic search results either from sagemaker.

    Args:
        chunks List[str]: a list of chunk strings

    Returns:
        results Dict[str, List[List[List[int, float]]]]: a dictionary
            where the keys are input sentences and the values are arrays of
            search results. For each sentence, an array of 5 results is
            returned.
    """
    try:
        sagemaker_runtime = boto3.client("sagemaker-runtime",
                                            region_name=region)
        data = {
            "vectors": vectors,
            "k": 5
        }
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=sagemaker_faiss_name,
            Body=bytes(json.dumps(data), 'utf-8'),
            ContentType='application/json'
        )
        raw_result: str = response['Body'].read().decode('utf-8')
        results: List[List[List[int, float]]] = json.loads(raw_result)
        return results
    except Exception as e:
        print(e)

In [ ]:
matches = get_matches(vectors)

matches

# 02/09/2023 - No2 Spacy

In [ ]:
import json
import requests
import pandas as pd
from typing import List

In [ ]:
df = pd.read_excel('soc_normalized_titles_matches_02092023.xlsx')
df

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

job_title_list = []
for job_title in df.Column1:
    doc = nlp(job_title)
    job_title_list.append(doc)

In [ ]:
job_title_list

In [ ]:
file_path = "C:/Users\KeikoGolden/ComputationalLiguistics/Annotaion/bn_nb_s/PythonTocheckModel/soc_cleanup.xlsx"
df1 = pd.read_excel(file_path)
df1.head(1)

In [ ]:
#Option 1
nlp = spacy.load('en_core_web_lg')

input_job_title_list = []
for jt in df1.final_title_post_processed:
    doc1 = nlp(str(jt))
    input_job_title_list.append(doc1)

In [ ]:
input_job_title_list

In [ ]:
# Option 2 - using df1 row - Do Not Use This
nlp = spacy.load('en_core_web_lg')

input_row_list = []
for row in df1.iterrows():
    doc1 = nlp(str(row))
    input_row_list.append(doc1)

In [ ]:
#Option1 -  Prints out blank cells if there is no match.
normalized = []
score = []
original = []
for x in job_title_list:
    for y in input_job_title_list:
        matches = x.similarity(y)
        if matches > 0.75:
            original.append(y)
            normalized.append(x)
            score.append(matches)
        if not matches:
            original.append(y)
            normalized.append(" ")
            score.append(" ")

In [ ]:
len(normalized)

In [ ]:
#Option2 - only the matched title will output/
normalized = []
score = []
original = []
for x in job_title_list:
    for y in input_job_title_list:
        matches = x.similarity(y)
        if matches > 0.75:
            original.append(y)
            normalized.append(x)
            score.append(matches)

In [ ]:
original[:25]

In [ ]:
normalized[:25]

In [ ]:
score[:25]

In [ ]:
results_df = pd.DataFrame({'minor_title': original, 'normalized_title': normalized, 'score': score})
results_df.head(10)

In [ ]:
results_df.to_excel('left_over_jt_normalized_by_spacy.xlsx')

In [ ]:
results_df.to_excel('left_over_jt_normalized_by_spacy_with_blank_cells.xlsx')

In [ ]:
results_df.minor_title.value_counts()

In [ ]:
for x in job_title_list:
    for y in input_job_title_list:
        matches = x.similarity(y)
        if matches > 0.75:
            print(y, '| ', x, '| ', matches)

# 02/11/2023 - SkyHive vectorizer

In [ ]:
minor_titles = df1.final_title_post_processed.to_csv("final_title_post_processed.csv")

In [ ]:
with open("final_title_post_processed.csv", 'r') as f:
    sample_desc = f.read()

In [ ]:
sample_desc

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')
doc = nlp(sample_desc)

In [ ]:
tokenized_sents = []
sents = []
for sent in doc.sents:
    sents.append(sent.text)
    tokens = [i.text for i in sent]
    tokenized_sents.append(tokens)

In [ ]:
tokenized_sents

In [ ]:
import boto3
import json
from typing import *

region = "ca-central-1"
vectorizer_name = "vectorizer-cpu-2"
sagemaker_runtime = boto3.client("sagemaker-runtime", region)

def vectorize_remote(
        sents: List[List[str]],
        offsets: List[List[List[int]]]) -> List[List[List[float]]]:
    data = {"sents": sents, "offsets": offsets}
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=vectorizer_name,
        Body=bytes(json.dumps(data), encoding="utf-8"),
        ContentType='application/json'

    )
    output = response['Body'].read().decode('utf-8')
    output = json.loads(json.loads(output)[0])
    return output

In [ ]:
offsets = [[[0, len(sent)]] for sent in tokenized_sents]
offsets

In [ ]:
#offsets = [[[0, len(i) + 2]] for i in tokenized_sents]
#offsets

In [ ]:
vectors = vectorize_remote(
    tokenized_sents,
    offsets
)

vectors

In [ ]:
sagemaker_faiss_name = "semantic-search-cpu-2"

def get_matches(vectors: List[List[List[float]]]) \
        -> List[List[List[float]]]:
    """Get semantic search results either from sagemaker.

    Args:
        chunks List[str]: a list of chunk strings

    Returns:
        results Dict[str, List[List[List[int, float]]]]: a dictionary
            where the keys are input sentences and the values are arrays of
            search results. For each sentence, an array of 5 results is
            returned.
    """
    try:
        sagemaker_runtime = boto3.client("sagemaker-runtime",
                                            region_name=region)
        data = {
            "vectors": vectors,
            "k": 5
        }
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=sagemaker_faiss_name,
            Body=bytes(json.dumps(data), 'utf-8'),
            ContentType='application/json'
        )
        raw_result: str = response['Body'].read().decode('utf-8')
        results: List[List[List[int, float]]] = json.loads(raw_result)
        return results
    except Exception as e:
        print(e)

In [ ]:
matches = get_matches(vectors)

matches

# 02/11/2023 - No2 Difflib

In [ ]:
file_path = "C:/Users\KeikoGolden/ComputationalLiguistics/Annotaion/bn_nb_s/PythonTocheckModel/soc_cleanup.xlsx"

In [ ]:
df = pd.read_excel(file_path)
df.head(1)

In [ ]:
df4 = pd.read_excel('soc_normalized_titles_matches_02092023.xlsx')
df4

In [ ]:
#Option1 -  Prints out blank cells if there is no match.

import difflib

normalized = []
score = []
original = []
for x in str(df4.Column1):
    for y in str(df.final_title_post_processed):
        matches = difflib.get_close_matches(y, x, n=5, cutoff=0.75)
        if matches:
            original.append(y)
            normalized.append(x)
            score.append(matches)
        if not matches:
            original.append(y)
            normalized.append(" ")
            score.append(" ")

In [ ]:
df = df.dropna(subset = ['occupation_title_working']).copy()

In [ ]:
df4 = df4.dropna(subset=['Column1']).copy()
df4

In [ ]:
#Option2 - only the matched title will output/
import difflib

normalized = []
score = []
original = []

for job_title in df['occupation_title_working']:
    matches = difflib.get_close_matches(job_title, df4['Column1'].tolist(), n=5, cutoff=0.65)
    if matches:
        original.append(job_title)
        normalized.append(matches)

In [ ]:
len(normalized)

In [ ]:
normalized

In [ ]:
results_df1 = pd.DataFrame({'minor_title': original, 'normalized_title': normalized})
results_df1.head(10)

In [ ]:
results_df1 = results_df1.explode('normalized_title')

In [ ]:
results_df1.to_excel('difflib_normalized.xlsx')

# 02/11/2023 Transformer Bert

In [ ]:
!pip install transformers

In [ ]:
pip install --upgrade pip

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings_minor_job_title = model.encode(df.final_title_post_processed.tolist())
sentence_embeddings_job_title = model.encode(df4.Column1)

In [ ]:
sentence_embeddings_minor_job_title.shape

In [ ]:
sentence_embeddings_job_title.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

minor_title = []
normlised_title = []
sim_score = []

for i, job_title in enumerate(sentence_embeddings_job_title):
    for j, title in enumerate(sentence_embeddings_minor_job_title):
        score = cosine_similarity([job_title],[title])
        if score > 0.80:
            minor_title.append(df.final_title_post_processed.iloc[j])
            normlised_title.append(df4.Column1.iloc[i])
            sim_score.append(score)

In [ ]:
minor_title[:25]

In [ ]:
len(minor_title)

In [ ]:
minor_title[:25]

In [ ]:
normlised_title[:25]

In [ ]:
bert_df_75 = pd.DataFrame({'minor_title': minor_title, 'normalized_title': normlised_title, 'score': sim_score})
bert_df_75.head(10)

In [ ]:
bert_df_80 = pd.DataFrame({'minor_title': minor_title, 'normalized_title': normlised_title, 'score': sim_score})
bert_df_80.head(10)

In [ ]:
bert_df_75.to_excel('BERT_75.xlsx')

In [ ]:
bert_df_80.to_excel('BERT_80.xlsx')

# 02/12/2023 - Using Original Detailed Title (occupation_title) with BERT

In [ ]:
df5 = pd.read_excel('soc_aligned.xlsx')
df5

In [ ]:
# normalize column names
df5 = df5.rename(columns={ i: "_".join(i.lower().split("-") ) for i in df5.columns })

In [ ]:
df5 = df5.dropna().copy()

In [ ]:
sentence_embeddings_minor_job_title1 = model.encode(df5.occupation_title.tolist())
sentence_embeddings_job_title1 = model.encode(df4.Column1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

minor_title = []
normlised_title = []
sim_score = []

for i, job_title in enumerate(sentence_embeddings_job_title1):
    for j, title in enumerate(sentence_embeddings_minor_job_title1):
        score = cosine_similarity([job_title],[title])
        if score > 0.75:
            minor_title.append(df5.occupation_title.iloc[j])
            normlised_title.append(df4.Column1.iloc[i])
            sim_score.append(score)

In [ ]:
bert_detailed_title_df_75 = pd.DataFrame({'occupation_title': minor_title, 'normalized_title': normlised_title, 'score': sim_score})
bert_detailed_title_df_75.head(10)

In [ ]:
def remove_brackets(x):
    res = str(x)[2:-2]
    return res

In [ ]:
bert_detailed_title_df_75['score'] = bert_detailed_title_df_75.score.apply(remove_brackets).copy()
bert_detailed_title_df_75['score']

In [ ]:
bert_detailed_title_df_75.head(10)

In [ ]:
bert_detailed_title_df_75.to_excel('bert_detailed_title75.xlsx')

# 02/16/2023 - Remaining 300+ skill extraction by BERT

In [ ]:
import json
import requests
import pandas as pd
from typing import List

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
remaing_jt_df = pd.read_excel('normalized_titles_unresolved_feb15.xlsx')
remaing_jt_df.head(3)

In [ ]:
detailed_jt_df = pd.read_excel('soc_cleanup.xlsx')
detailed_jt_df.head(3)

In [ ]:
vector_detialed_jt = model.encode(detailed_jt_df.final_title_post_processed.tolist())
vector_ramaining_jt = model.encode(remaing_jt_df.normalized_title)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

detailed_title = []
normlised_title = []
sim_score = []

for i, job_title in enumerate(vector_ramaining_jt):
    for j, title in enumerate(vector_detialed_jt):
        score = cosine_similarity([job_title],[title])
        if score > 0.75:
            detailed_title.append(detailed_jt_df.final_title_post_processed.iloc[j])
            normlised_title.append(remaing_jt_df.normalized_title.iloc[i])
            sim_score.append(score)

In [ ]:
remaining_300_normalized = pd.DataFrame({'detailed_title': detailed_title, 'normalized_title': normlised_title, 'score': sim_score})
remaining_300_normalized.head(3)

In [ ]:
def remove_brackets(x):
    res = str(x)[2:-2]
    return res

In [ ]:
remaining_300_normalized['score'] = remaining_300_normalized.score.apply(remove_brackets).copy()
remaining_300_normalized['score'][:4]

In [ ]:
remaining_300_normalized.head(3)

In [ ]:
remaining_300_normalized.normalized_title.value_counts()

In [ ]:
len(remaining_300_normalized)

In [ ]:
remaining_300_normalized.to_excel('remaining_300_soc_normalized1.xlsx')

In [ ]:
unique_values = remaining_300_normalized.normalized_title.unique()
len(unique_values)

In [ ]:
remainder = []
for row in remaing_jt_df.normalized_title.tolist():
    if row not in unique_values:
        remainder.append(row)
        #print(row)

In [ ]:
len(remainder)

In [ ]:
remainder

In [ ]:
remainder_df = pd.DataFrame({'remainder':remainder})

In [ ]:
remainder_df.to_excel('remainder.xlsx')

### ここから下は無視してオーケー。SkyHiveのパイプラインよりBERTの方が優れていることをアビゲルに証明するために同じデータでアウトプット。

#### オプション１

In [ ]:
import json
import requests
import pandas as pd
from typing import List

In [ ]:
def normalize_v3(skill_list, k=5):
    response = requests.post(
    url = 'https://testazure-ml-api.skyhive.io/job-title-normalization/titles_v3',
    json = {"titles": skill_list, "k": k}
)
    return response.json()

In [ ]:
from tqdm import tqdm
def main_v3(skills: List[str]):
    responses = []

    for skill in tqdm(skills):
        try:
            response = normalize_v3([skill])
        except:
            response = None
        responses.append(response)
    return responses

In [ ]:
remaing_jt_df = pd.read_excel('normalized_titles_unresolved_feb15.xlsx')
remaing_jt_df.head(3)

In [ ]:
occupation_title = remaing_jt_df['normalized_title']
occupation_title

In [ ]:
responses = main_v3(occupation_title)
responses

In [ ]:
###WIP
keywords_list = []
new_normalized = []
score = []

for response in responses:
    for i, keywords in enumerate(response):
        for job_title in keywords["Results"]: 
            new_normalized.append(job_title['categoryName']) 
            score.append(job_title["score"]) 

In [ ]:
keywords_list = []
new_normalized = []
score = []

for response in responses:
    for keywords in response:
        for job_title in keywords["Results"]: 
            new_normalized.append(job_title['categoryName']) 
            score.append(job_title["score"]) 

In [ ]:
new_normalized

In [ ]:
score

In [ ]:
new_normalized_df = pd.DataFrame()

In [ ]:
new_normalized_df['new_normalized_expanded'] = new_normalized

In [ ]:
new_normalized_df['score'] = score

In [ ]:
new_normalized_df.head(5)

In [ ]:
new_normalized_df.to_excel('skill_normalization_for_occupation_annotation.xlsx')

### オプション２

In [ ]:
import json
import requests
import pandas as pd
from typing import List
import vector_searc

In [ ]:
vs = VectorSearch()